In [1]:
import requests
from bs4 import BeautifulSoup
import os
import fitz

In [2]:
def parse_base_html():
    base_url = "https://proceedings.neurips.cc/paper_files/paper/2023"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paper_li = soup.find_all('li', class_='conference')
    paper_links = []
    for i in range(len(paper_li)):
        paper_a = paper_li[i].find('a')
        paper_links.append(paper_a.get('href'))
    modify_links(paper_links)

In [3]:
def modify_links(paper_links):
    paper_urls = []
    for link in paper_links:
        paper_url = "https://proceedings.neurips.cc" + \
                    link.replace("-Abstract-", "-Paper-")\
                        .replace("/hash/", "/file/")\
                        .replace(".html", ".pdf")
        paper_urls.append(paper_url)
    return paper_urls

TypeError: 'NoneType' object is not subscriptable